# Chapter 2 - Exercise 5: Nutrient Diffusion and Cell Viability

## Understanding Mass Transport Limitations in Scaffold-Free Biofabrication

**Learning Objectives:**
- Model oxygen and nutrient gradients in spheroids and 3D constructs
- Understand formation of necrotic cores due to diffusion limitations
- Calculate critical size limits for scaffold-free constructs
- Explore how vascularization improves nutrient delivery

**Python Skills:**
- Solving differential equations (simplified diffusion models)
- Creating 2D contour plots and gradient visualizations
- Animating time-dependent processes

---

## Setup and Imports

Run this cell first to install required packages and import libraries.

In [ ]:
# Install required packages
!pip install numpy matplotlib scipy pandas seaborn ipywidgets

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import pandas as pd
import seaborn as sns
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve
import ipywidgets as widgets
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 12

print("✅ All packages imported successfully!")
print("📚 Ready to explore nutrient diffusion in scaffold-free constructs!")

## 📖 Background: Diffusion Limitations in 3D Tissue Constructs

### The Diffusion Problem

In scaffold-free biofabrication, cells must rely on **diffusion** to receive oxygen and nutrients. This creates fundamental limitations:

- **Oxygen** has low solubility in aqueous media (~0.2 mM at 37°C)
- **Diffusion** is relatively slow over distances >100 μm
- **Cell consumption** creates gradients that worsen with construct size
- **Necrotic cores** form when oxygen drops below survival thresholds

### Mathematical Model

For a spherical construct, oxygen concentration C(r,t) follows:

**Fick's Second Law in Spherical Coordinates:**
```
∂C/∂t = D * (∂²C/∂r² + (2/r) * ∂C/∂r) - R_max * C/(K_m + C)
```

Where:
- D = diffusion coefficient (cm²/s)
- R_max = maximum consumption rate (mM/s)
- K_m = Michaelis-Menten constant (mM)
- C = oxygen concentration (mM)

### Critical Parameters

| Parameter | Typical Value | Units |
|-----------|---------------|-------|
| D_oxygen | 2.4 × 10⁻⁵ | cm²/s |
| R_max | 1.0 × 10⁻⁸ | mM/s |
| K_m | 0.01 | mM |
| C_critical | 0.01 | mM |
| C_surface | 0.2 | mM |

## 🗃️ Tissue Properties Database

Different cell types have varying oxygen consumption rates and critical thresholds.

In [ ]:
# Tissue properties database
tissue_properties = {
    'Hepatocytes': {
        'R_max': 2.5e-8,  # mM/s - high metabolism
        'K_m': 0.015,     # mM
        'C_critical': 0.02,  # mM - sensitive to hypoxia
        'cell_density': 1e8,  # cells/cm³
        'description': 'High oxygen consumption, sensitive to hypoxia'
    },
    'Cardiomyocytes': {
        'R_max': 2.0e-8,  # mM/s
        'K_m': 0.012,     # mM
        'C_critical': 0.015, # mM
        'cell_density': 8e7,  # cells/cm³
        'description': 'High energy demand, critical for heart function'
    },
    'Chondrocytes': {
        'R_max': 0.5e-8,  # mM/s - low metabolism
        'K_m': 0.008,     # mM
        'C_critical': 0.005, # mM - hypoxia tolerant
        'cell_density': 2e7,  # cells/cm³
        'description': 'Low oxygen consumption, hypoxia tolerant'
    },
    'Tumor_cells': {
        'R_max': 1.5e-8,  # mM/s
        'K_m': 0.010,     # mM
        'C_critical': 0.008, # mM - somewhat tolerant
        'cell_density': 1.2e8, # cells/cm³
        'description': 'Variable metabolism, can survive low oxygen'
    },
    'Neural_cells': {
        'R_max': 1.8e-8,  # mM/s
        'K_m': 0.020,     # mM
        'C_critical': 0.025, # mM - very sensitive
        'cell_density': 6e7,  # cells/cm³
        'description': 'High oxygen demand, extremely sensitive to hypoxia'
    },
    'Fibroblasts': {
        'R_max': 0.8e-8,  # mM/s - moderate metabolism
        'K_m': 0.009,     # mM
        'C_critical': 0.010, # mM
        'cell_density': 5e7,  # cells/cm³
        'description': 'Moderate consumption, relatively robust'
    }
}

# Physical constants
constants = {
    'D_oxygen': 2.4e-5,    # cm²/s - oxygen diffusion coefficient
    'C_surface': 0.2,      # mM - surface oxygen concentration
    'temp': 37,            # °C - physiological temperature
}

# Display tissue properties
print("🧬 Tissue Properties Database")
print("=" * 50)
df_tissues = pd.DataFrame(tissue_properties).T
display(df_tissues[['R_max', 'C_critical', 'cell_density', 'description']])